In [4]:
# Clone YOUR fork
!cd /content && rm -rf bd3lms
!cd /content && git clone https://github.com/ntua-el21050/bd3lms.git

!mkdir -p /content/bd3lms/data
!mkdir -p /content/repro_runs
!mkdir -p /content/hf_checkpoints
!mkdir -p /content/sample_logs

Cloning into 'bd3lms'...
remote: Enumerating objects: 1223, done.
remote: Counting objects: 100% (486/486), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 1223 (delta 361), reused 332 (delta 248), pack-reused 737 (from 2)
Receiving objects: 100% (1223/1223), 8.76 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (770/770), done.


In [5]:
!pip install -q torchmetrics==1.6.2 datasets==3.3.2 einops==0.8.1 \
    hydra-core==1.3.2 lightning==2.5.0.post0 transformers==4.49.0 \
    huggingface_hub fsspec==2024.2.0 omegaconf==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 134.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import subprocess
import re
import os
import shutil
import sys
import torch
import json
import numpy as np
from pathlib import Path

sys.path.insert(0, '/content/bd3lms')

def run_main(overrides, timeout=None):
    """Run `python -u main.py ...` and return combined stdout/stderr text."""
    env = dict(os.environ)
    env.setdefault("HYDRA_FULL_ERROR", "1")
    cmd = [sys.executable, "-u", "/content/bd3lms/main.py", *overrides]
    print("\n$", " ".join(cmd))
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        timeout=timeout,
        check=False,
        env=env,
    )
    print(proc.stdout[-4000:])  # tail for quick visibility
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with return code {proc.returncode}")
    return proc.stdout

_METRIC_PATTERNS = [
    # Key: value (some loggers print this)
    re.compile(r"val/ppl\s*[:=]\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),
    re.compile(r"'val/ppl'\s*:\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),

    # Lightning "rich" table row (note the unicode box character │)
    re.compile(r"val/ppl\s*[│|]\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),
]

# def extract_val_ppl(log_text: str):
#     # First try line-based parse from the end (most reliable for tables)
#     for line in reversed(log_text.splitlines()):
#         if "val/ppl" in line.lower():
#             m = re.search(r"val/ppl.*?([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", line, re.IGNORECASE)
#             if m:
#                 return float(m.group(1))

#     # Fallback: scan entire text with known patterns
#     hits = []
#     for pat in _METRIC_PATTERNS:
#         hits.extend(pat.findall(log_text))
#     return float(hits[-1]) if hits else None

def strip_ansi(text):
    """Αφαιρεί τους κρυφούς χαρακτήρες χρώματος/μορφοποίησης (ANSI codes)."""
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

def extract_val_ppl(log_text: str):
    # 1. Καθαρίζουμε το κείμενο από χρώματα και ειδικούς χαρακτήρες
    clean_text = strip_ansi(log_text)

    # 2. Ψάχνουμε από το τέλος προς την αρχή (για να βρούμε το τελικό αποτέλεσμα)
    for line in reversed(clean_text.splitlines()):
        # Εντοπισμός της γραμμής που περιέχει το metric
        if "val/ppl" in line.lower():
            # Ψάχνουμε για τον αριθμό (float) που ακολουθεί το "val/ppl"
            # Το .*? επιτρέπει να υπάρχουν οσοιδήποτε χαρακτήρες (π.χ. │ ή κενά) ενδιάμεσα
            m = re.search(r"val/ppl.*?([0-9]+\.[0-9]+(?:e[+-]?\d+)?)", line, re.IGNORECASE)
            if m:
                return float(m.group(1))

    # Fallback σε περίπτωση που δεν βρεθεί στη γραμμική σάρωση
    print("Warning: Could not parse ppl from line, trying patterns...")
    return 0.0

def _small_loader_overrides(batch_size=8, num_workers=2):
    """Overrides needed to avoid huge default batch sizes on Colab."""
    return [
        f"loader.global_batch_size={batch_size}",
        f"loader.eval_global_batch_size={batch_size}",
        f"loader.batch_size={batch_size}",
        f"loader.eval_batch_size={batch_size}",
        f"loader.num_workers={num_workers}",
        "trainer.accumulate_grad_batches=1",
    ]

def train_run(run_name, algo, block_size=None, from_pretrained=None, max_steps=800, extra_overrides=None):
    """Train a model (optionally from a base checkpoint) and return the last.ckpt path."""
    save_dir = (Path("/content/repro_runs") / run_name).resolve()
    #save_dir = Path("/content/repro_runs") / run_name
    if save_dir.exists():
        shutil.rmtree(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    overrides = [
        "mode=train",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        "data.max_train_samples=5000",
        # For LM1B, validation uses the 'test' split in this codebase
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "trainer.log_every_n_steps=10",
        "trainer.val_check_interval=50",
        f"trainer.max_steps={max_steps}",
        "data.max_valid_samples=1000",
        "data.max_test_samples=1000",
        f"checkpointing.save_dir={str(save_dir)}",
        "checkpointing.resume_from_ckpt=false",
        "wandb=null",
    ]
    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))
    if block_size is not None:
        overrides.append(f"block_size={block_size}")
    if from_pretrained is not None:
        overrides.append(f"training.from_pretrained={Path(from_pretrained).resolve()}")
    if extra_overrides:
        overrides.extend(extra_overrides)

    _ = run_main(overrides)
    ckpt = save_dir / "checkpoints" / "best.ckpt"
    if not ckpt.exists():
        raise FileNotFoundError(f"Expected checkpoint not found: {ckpt}")
    return str(ckpt)

def eval_run(algo, checkpoint_path, block_size=None, extra_overrides=None):
    """Evaluate perplexity (val/ppl) for a given checkpoint."""
    overrides = [
        "mode=ppl_eval",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        # For LM1B, `get_dataloaders` maps validation to the 'test' split
        "data.max_test_samples=1000",
        "data.max_valid_samples=1000",
        "data.max_train_samples=5000",
        "+data.skip_samples=0",
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        f"eval.checkpoint_path={checkpoint_path}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "wandb=null",
    ]
    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))
    if block_size is not None:
        overrides.append(f"block_size={block_size}")
    if extra_overrides:
        overrides.extend(extra_overrides)

    log_text = run_main(overrides)
    ppl = extract_val_ppl(log_text)
    if ppl is None:
        raise ValueError("Could not parse val/ppl from output. Try increasing the log tail or printing full logs.")
    return ppl

In [7]:
import os
import shutil
import sys

results = []

bd3lm_base_run = "bd3lm_base_len128"
bd3lm_base_ckpt = train_run(
    bd3lm_base_run,
    algo="bd3lm",
    block_size=128,
    extra_overrides=[
        "training.resample=false",
        "algo.var_min=false",
        "algo.clip_search_widths=[]",
    ],
)

for Lprime in [16, 8, 4]:
    finetune_run = f"bd3lm_finetune_Lp{Lprime}"
    finetune_ckpt = train_run(
        finetune_run,
        algo="bd3lm",
        block_size=Lprime,
        from_pretrained=bd3lm_base_ckpt,
        extra_overrides=[
            "training.resample=true",
            "algo.var_min=false",
            "algo.clip_search_widths=[]",
        ],
    )
    ppl = eval_run(
        algo="bd3lm",
        checkpoint_path=finetune_ckpt,
        block_size=Lprime,
        extra_overrides=[
            "algo.var_min=false",
        ],
    )
    results.append({"model": "Block diffusion (BD3LM)", "block_size_Lprime": Lprime, "val_ppl": ppl})


$ /usr/bin/python3 -u /content/bd3lms/main.py mode=train data=lm1b-wrap data.cache_dir=/content/bd3lms/data data.streaming=true data.max_train_samples=5000 model=tiny model.length=128 model.attn_backend=sdpa algo=bd3lm trainer.accelerator=gpu trainer.devices=1 trainer.num_nodes=1 trainer.precision=16-mixed trainer.num_sanity_val_steps=0 trainer.log_every_n_steps=10 trainer.val_check_interval=50 trainer.max_steps=800 data.max_valid_samples=1000 data.max_test_samples=1000 checkpointing.save_dir=/content/repro_runs/bd3lm_base_len128 checkpointing.resume_from_ckpt=false wandb=null loader.global_batch_size=8 loader.eval_global_batch_size=8 loader.batch_size=8 loader.eval_batch_size=8 loader.num_workers=2 trainer.accumulate_grad_batches=1 block_size=128 training.resample=false algo.var_min=false algo.clip_search_widths=[]
--------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
-------------------------------

In [8]:
def print_table(rows):
    if not rows:
        print("No data to display.")
        return

    columns = list(rows[0].keys())

    str_rows = [
        {col: str(row.get(col, "")) for col in columns}
        for row in rows
    ]

    widths = {
        col: max(len(col), max(len(row[col]) for row in str_rows))
        for col in columns
    }

    def print_separator():
        print("+" + "+".join("-" * (widths[col] + 2) for col in columns) + "+")

    def print_row(row):
        print(
            "| " +
            " | ".join(row[col].ljust(widths[col]) for col in columns) +
            " |"
        )

    # Print table
    print_separator()
    print_row({col: col for col in columns})
    print_separator()
    for row in str_rows:
        print_row(row)
    print_separator()

print_table(results)

+-------------------------+-------------------+--------------------+
| model                   | block_size_Lprime | val_ppl            |
+-------------------------+-------------------+--------------------+
| Block diffusion (BD3LM) | 16                | 1405.5001220703125 |
| Block diffusion (BD3LM) | 8                 | 1203.98974609375   |
| Block diffusion (BD3LM) | 4                 | 1204.0213623046875 |
+-------------------------+-------------------+--------------------+
